In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

import random
import time
import json

from nasbench.lib import cifar
from nasbench.lib import model_builder
from nasbench.lib import training_time
import numpy as np


from nasbench.lib import evaluate
from nasbench.lib import model_spec
from nasbench.lib import config as _config

from nasbench.scripts.run_evaluation import NumpyEncoder

RESULTS_FILE = 'results.json'

In [2]:
from nasbench import api

nasbench = api.NASBench('../data/nasbench_only108.tfrecord')

Loading dataset from file... This may take a few minutes...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Loaded dataset in 95 seconds


In [3]:
keys = random.sample(nasbench.fixed_statistics.keys(), 20)

In [4]:
import json
with open("../data/student_data/keys_20.json", 'w') as f:
    json.dump(keys, f)

In [8]:
import tqdm.notebook

accs = []
for key in keys:
    acc = np.mean([s['final_test_accuracy'] for s in nasbench.get_metrics_from_hash(key)[1][108]])
    accs.append((key, acc))

In [10]:
sorted(accs, key=lambda x: x[1])

[('9b1e8664851aa960d440b075e68583f6', 0.842848559220632),
 ('9e25f7ad3d1878b2032537ca1848904f', 0.8656183083852133),
 ('dfdeefbd128a1268fb4712b208ca9118', 0.8774038354555765),
 ('69348c1581d0b2a913ab1b0630f8bfc0', 0.8778712550799052),
 ('3e06877960ce32d0f1313f275a2d1561', 0.8833132982254028),
 ('e3c41bd2916fea4657056b77eed71adc', 0.8833132982254028),
 ('2ad4b83cbd27a532f99dbb67a939dcaf', 0.8930622339248657),
 ('7f2e534b633c971e639fd00f363fdd05', 0.8977363705635071),
 ('d6e80a86732ec73ef9f6b2c4ddd603f8', 0.9027777512868246),
 ('5595a4314b345d232f59c6140cbb8bf5', 0.9037125905354818),
 ('cf690b677beb7c45f63cab214fab8616', 0.9055154919624329),
 ('ac8193cd15dcbefe09a85d8e21936ab9', 0.9066840211550394),
 ('e880a849a4aaa64be78b59e899663df0', 0.9141626358032227),
 ('34cebf6f6c3e1b953edd782b69f850fc', 0.9153979818026224),
 ('8f53b0fcabc01639cd01c76b883c5855', 0.9193376104036967),
 ('5e710c5eacb1c1749d91600baa81915c', 0.9204727411270142),
 ('41fb42943850e7faf7746c0ca98a6f52', 0.9214076002438863)

In [10]:
nasbench.fixed_statistics['02e5a0247bbdcf2860b7e96f74961594']

{'module_adjacency': array([[0, 1, 1, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0]], dtype=int8),
 'module_operations': ['input',
  'conv1x1-bn-relu',
  'conv3x3-bn-relu',
  'maxpool3x3',
  'conv3x3-bn-relu',
  'conv3x3-bn-relu',
  'output'],
 'trainable_parameters': 32426634}

In [13]:
nasbench.fixed_statistics['02e5a0247bbdcf2860b7e96f74961594']

{'module_adjacency': array([[0, 1, 1, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0]], dtype=int8),
 'module_operations': ['input',
  'conv1x1-bn-relu',
  'conv3x3-bn-relu',
  'maxpool3x3',
  'conv3x3-bn-relu',
  'conv3x3-bn-relu',
  'output'],
 'trainable_parameters': 32426634}

In [3]:
cfg = _config.build_config()

In [11]:
cfg

{'train_data_files': [],
 'valid_data_file': '',
 'test_data_file': '',
 'sample_data_file': '',
 'data_format': 'channels_last',
 'num_labels': 10,
 'module_vertices': 7,
 'max_edges': 9,
 'available_ops': ['conv3x3-bn-relu', 'conv1x1-bn-relu', 'maxpool3x3'],
 'stem_filter_size': 128,
 'num_stacks': 3,
 'num_modules_per_stack': 3,
 'batch_size': 256,
 'train_epochs': 108,
 'train_seconds': 14400.0,
 'learning_rate': 0.1,
 'lr_decay_method': 'COSINE_BY_STEP',
 'momentum': 0.9,
 'weight_decay': 0.0001,
 'max_attempts': 5,
 'intermediate_evaluations': ['0.5'],
 'num_repeats': 3,
 'use_tpu': True,
 'tpu_iterations_per_loop': 100,
 'tpu_num_shards': 2}

In [16]:
from pathlib import Path

In [28]:
list(map(str, list(Path.cwd().glob("../data/dataset/train_*.tfrecords"))))

['/notebooks/nasbench/scripts/../data/dataset/train_3.tfrecords',
 '/notebooks/nasbench/scripts/../data/dataset/train_2.tfrecords',
 '/notebooks/nasbench/scripts/../data/dataset/train_4.tfrecords',
 '/notebooks/nasbench/scripts/../data/dataset/train_1.tfrecords']

In [31]:
Path("../data", "*", "123")

PosixPath('../data/*/123')

In [32]:
import json
import pathlib

paths = pathlib.Path('../data/student_data/students.t_1.lmb_0.5/').glob('student_*')
for path in paths:
    with open(pathlib.Path(path, "meta.json"), 'r') as f:
        meta = json.load(f)
    print(meta['evaluation_results'][-1]['training_time'] / 60 / 60, 'h')

2.6720016818576386 h
1.1845109049479168 h
3.7227913411458333 h
1.7947283257378472 h
1.3620741102430556 h
1.5089516872829862 h


FileNotFoundError: [Errno 2] No such file or directory: '../data/student_data/students.t_1.lmb_0.5/student_f7ea2d1d5b7bfad603ca2955ea0945f7/meta.json'

In [2]:
!ls ../data/dataset/train_1_KD_11.tfrecords

../data/dataset/train_1_KD_11.tfrecords


In [3]:
import tensorflow as tf
tf.enable_eager_execution()
tf.get_logger().setLevel('ERROR')

In [7]:
file = "../data/dataset/train_1_KD_11.tfrecords"
raw_dataset = tf.data.TFRecordDataset([file])

In [8]:
i = 0
for r in raw_dataset:
    i += 1
print(i)

1111


In [9]:
10000 * (11 / 100.0)

1100.0

In [ ]:
key = next(nasbench.fixed_statistics.keys())

In [ ]:
key 